<center><h1>Wong_Jonathan_HW3</h1></center>
<br>
<br>

Name: Jonathan Wong
<br>
Github Username: jonathanwong150
<br>
USC ID: 9171949586

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [76]:
import os
import pandas as pd
import numpy as np
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

Get the AReM Data Set

In [77]:
# Initialize empty lists to hold test and train data
testData = []
trainData = []

# List of folders in the dataset
folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']

# Define the path to the dataset
dataPath = "../data/AReM/"

### (b) Test and Train Data

In [78]:
# Loop through each folder to read the CSV files
for folder in folders:
    folderPath = os.path.join(dataPath, folder)
    files = os.listdir(folderPath)
    
    # Sort files to ensure they are in numerical order
    files.sort()
    
    # Decide which files to use for training and testing based on the task description
    if folder in ['bending1', 'bending2']:
        testFiles = files[:2]
        trainFiles = files[2:]
    else:
        testFiles = files[:3]
        trainFiles = files[3:]
    
    # Read test and train files and append them to respective lists
    for file in testFiles:
        filePath = os.path.join(folderPath, file)
        df = pd.read_csv(filePath, skiprows=4)
        df['label'] = folder
        testData.append(df)
    
    for file in trainFiles:
        filePath = os.path.join(folderPath, file)
        df = pd.read_csv(filePath, skiprows=4)
        df['label'] = folder
        trainData.append(df)

# Convert lists to pandas DataFrames
testData = pd.concat(testData, keys=range(len(testData)))
trainData = pd.concat(trainData, keys=range(len(trainData)))

### (c) Feature Extraction

#### i. Research

Statistical Features:
- Minimum
- Maximum
- Mean
- Median
- Standard Deviation
- Variance
- Skewness
- Kurtosis
- Quartile 1
- Quartile 3
- IQR

Time Series Features:
- Autocorrelation
- Cross-correlation

#### ii. Extraction

In [79]:
# Initialize empty lists to hold feature vectors for test and train data
testFeatures = []
trainFeatures = []

# List of columns that represent time series in the dataset
seriesCols = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

# Function to calculate time-domain features for each instance
def calculateFeatures(data):
    features = []
    for column in seriesCols:
        minVal = np.min(data[column])
        maxVal = np.max(data[column])
        meanVal = np.mean(data[column])
        medianVal = np.median(data[column])
        std = np.std(data[column])
        firstQuart = np.percentile(data[column], 25)
        thirdQuart = np.percentile(data[column], 75)
        
        features.extend([minVal, maxVal, meanVal, medianVal, std, firstQuart, thirdQuart])
    return features

# Loop through each instance in test and train data to calculate features
for i in range(len(testData.groupby(level=0))):
    instance = testData.loc[i]
    features = calculateFeatures(instance)
    features.append(instance['label'].iloc[0])
    testFeatures.append(features)

for i in range(len(trainData.groupby(level=0))):
    instance = trainData.loc[i]
    features = calculateFeatures(instance)
    features.append(instance['label'].iloc[0])
    trainFeatures.append(features)

# Convert feature lists to DataFrames and add column names
featureCols = [f"{feature}{i+1}" for i in range(6) for feature in ['min_', 'max_', 'mean_', 'median_', 'std_dev_', '1st_quart_', '3rd_quart_']]
featureCols.append('label')

testFeatDf = pd.DataFrame(testFeatures, columns=featureCols)
trainFeatDf = pd.DataFrame(trainFeatures, columns=featureCols)

# Output the new dataset with extracted features
print("Test Features DataFrame:")
print(testFeatDf.head())

print("\nTrain Features DataFrame:")
print(trainFeatDf.head())


Test Features DataFrame:
   min_1  max_1     mean_1  median_1  std_dev_1  1st_quart_1  3rd_quart_1  \
0  37.25  45.00  40.624792     40.50   1.475428      39.2500        42.00   
1  38.00  45.67  42.812812     42.50   1.434054      42.0000        43.67   
2  12.75  51.00  24.562958     24.25   3.733619      23.1875        26.50   
3   0.00  42.75  27.464604     28.00   3.579847      25.5000        30.00   
4  24.25  45.00  37.177042     36.25   3.577569      34.5000        40.25   

   min_2  max_2    mean_2  ...  1st_quart_5  3rd_quart_5  min_6  max_6  \
0    0.0   1.30  0.358604  ...        33.00        36.00    0.0   1.92   
1    0.0   1.22  0.372437  ...        32.00        34.50    0.0   3.11   
2    0.0   6.87  0.590833  ...        20.50        27.00    0.0   4.97   
3    0.0   7.76  0.449708  ...        15.00        20.75    0.0   6.76   
4    0.0   8.58  2.374208  ...        17.95        21.75    0.0   9.34   

     mean_6  median_6  std_dev_6  1st_quart_6  3rd_quart_6     labe

#### iii. Standard Deviation

In [80]:
combinedDf = pd.concat([trainFeatDf, testFeatDf])

# Features to consider
domainFeatures = ['min_', 'max_', 'mean_', 'median_', 'std_dev_', '1st_quart_', '3rd_quart_']
timeSeries = [str(i) for i in range(1, 7)]

# Mapping from your DataFrame column names to desired output names
dic = {
    "1": "avg_rss12",
    "2": "var_rss12",
    "3": "avg_rss13",
    "4": "var_rss13",
    "5": "avg_rss23",
    "6": "var_rss23",
}

# Initialize the results list
results = []

# Loop over each feature for each time series
for series in timeSeries:
    for feature in domainFeatures:
        colName = f"{feature}{series}"
        
        # Check if the column exists in the DataFrame
        if colName in combinedDf.columns:
        
            # Calculate standard deviation
            std = combinedDf[colName].std()

            # Bootstrap 90% confidence interval for standard deviation using bootstrapped library
            ci = bs.bootstrap(combinedDf[colName].values, stat_func=bs_stats.std, alpha=0.1)
            low = ci.lower_bound
            high = ci.upper_bound

            # Convert to desired names and round to 3 decimal places
            mapped_name = f"{feature}{dic[series]}"
            std = round(std, 3)
            low = round(low, 3)
            high = round(high, 3)

            # Append to results
            results.append({
                "Feature Name": mapped_name,
                "Standard Deviation": std,
                "Lower Bound": low,
                "Upper Bound": high
            })
        else:
            print(f"Column {colName} not found in DataFrame.")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Print the results in the desired format
print("\t\t\t    Standard Deviation   Lower Bound   Upper Bound")
print("+=========================+====================+=============+============+")
for index, row in results_df.iterrows():
    print(f"| {row['Feature Name']:<23} | {row['Standard Deviation']:<18} | {row['Lower Bound']:<11} | {row['Upper Bound']:<11} |")
    print("+-------------------------+--------------------+-------------+------------+")

			    Standard Deviation   Lower Bound   Upper Bound
+=========================+====================+=============+============+
| min_avg_rss12           | 9.57               | 8.29        | 10.804      |
+-------------------------+--------------------+-------------+------------+
| max_avg_rss12           | 4.394              | 3.472       | 5.447       |
+-------------------------+--------------------+-------------+------------+
| mean_avg_rss12          | 5.336              | 4.771       | 5.931       |
+-------------------------+--------------------+-------------+------------+
| median_avg_rss12        | 5.44               | 4.857       | 6.05        |
+-------------------------+--------------------+-------------+------------+
| std_dev_avg_rss12       | 1.77               | 1.584       | 1.955       |
+-------------------------+--------------------+-------------+------------+
| 1st_quart_avg_rss12     | 6.154              | 5.639       | 6.706       |
+-------------------------+-

#### iv. Select Features

- Mean: The best measure of central tendency and used in a lot of our other calculations.
- Median: Strong measure of central tendency that is resistant to outliers.
- Standard Deviation: Extremely helpful to better understand the spread and estimation of data.

## 2. ISLR 3.7.4

### (a) Linear Train

The cubic regression would most likely have a lower RSS than the linear regression because it could make a tighter fit and is more flexible than the linear regression.

### (b) Linear Test

The linear regression would have a lower RSS as the cubic regression since it will be overfit from the training data.

### (c) Not Linear Train

The cubic regression would again have a lower RSS in this case because the more flexible model will be able to align with the data points more closely.

### (d) Not Linear Testing

We do not have enough information to determine which would have a lower RSS. To determine this, we would need to know how far the true relationship is from linear or cubic. Whichever one it is closest to, will have the lower RSS.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 